In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
import pandas as pd

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')  # Download the WordNet corpus

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Create an instance of the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
pip install spacy

In [ ]:
!python -m spacy download en

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy

# Load the English language model
nlp = spacy.load('en_core_web_sm')

In [ ]:
df = pd.read_csv('edge_list.csv')
df.head()

In [ ]:
unique_topics = df['topics'].unique()
print(unique_topics)

In [ ]:
import ast

# Convert the string representation of the list to a list
df['topics'] = df['topics'].apply(ast.literal_eval)


In [ ]:
# Function to extract the most repeated word from a list of words
def get_most_repeated_word(words):
    lemmatized_words = [token.lemma_ for token in nlp(' '.join(words))]
    word_counts = Counter(lemmatized_words)
    most_common_word, count = word_counts.most_common(1)[0]
    return most_common_word if count > 1 else lemmatized_words[0]

In [ ]:
import pandas as pd
from collections import Counter
from collections import defaultdict
from collections import defaultdict, Counter

In [ ]:

# Define the updated get_most_repeated_word() function
def get_most_repeated_word(words, assigned_words):
    lemmatized_words = [token.lemma_ for token in nlp(' '.join(words))]
    word_counts = Counter(lemmatized_words)

    for word, count in word_counts.most_common():
        if count > 1:
            root_word = word
            if root_word not in assigned_words:
                return root_word

    for word in words:
        if word not in assigned_words:
            return word

    # If all words are already assigned, assign a unique repeated word with a numbering scheme
    numbering = 1
    while True:
        repeated_word = f"{words[0]}_{numbering}"
        if repeated_word not in assigned_words:
            return repeated_word
        numbering += 1

    # If no unique repeated word can be assigned, return None
    return None

# Assuming df is your DataFrame
assigned_words = set()
df['topic_title'] = df['topics'].apply(lambda x: get_most_repeated_word(x, assigned_words) if isinstance(x, list) else None)

# Update the assigned_words set with the newly assigned repeated words
assigned_words.update(df['topic_title'].dropna())

# Print the DataFrame to verify the changes
print(df)


In [ ]:
# Group the DataFrame by 'repeated_word' and apply a lambda function to create a set of unique topics for each repeated word
df_duplicates = df.groupby('topic_title')['topics'].apply(lambda x: set(tuple(topic) for topic in x)).reset_index()

# Calculate the number of unique topics for each repeated word and add it as a new column 'num_unique_topics'
df_duplicates['num_unique_topics'] = df_duplicates['topics'].apply(len)

# Filter out rows where the number of unique topics is greater than 1, and get the list of duplicated repeated words
duplicated_repeated_words = df_duplicates[df_duplicates['num_unique_topics'] > 1]['topic_title'].tolist()

# Print the list of duplicated repeated words
print(duplicated_repeated_words)


In [ ]:
# Define a dictionary to store the unique words for each topic
topic_repeated_words = {
    ('2050', '2030s', 'catastrophic', 'decades', '2040s', '2060', '2045', 'famine', 'billion', '2040'): 'catastrophe',
    #add more topics and topic titles
}

def update_repeated_word(row):
    topics = tuple(row['topics'])  # Convert the list to a tuple
    repeated_word = row['topic_title']
    
    # Update the repeated word based on the topic using the dictionary
    if topics in topic_repeated_words:
        repeated_word = topic_repeated_words[topics]
    
    # Return the updated repeated word
    return repeated_word

# Apply the function to update the repeated word column
df['topic_title'] = df.apply(update_repeated_word, axis=1)


In [ ]:
df.to_csv('edge_list.csv', index=False)